In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/data

/content/drive/MyDrive/data


In [ ]:
%%shell

wget https://www.cnts.ua.ac.be/conll2000/chunking/train.txt.gz
gzip -d train.txt.gz

--2023-03-14 00:21:38--  https://www.cnts.ua.ac.be/conll2000/chunking/train.txt.gz
Resolving www.cnts.ua.ac.be (www.cnts.ua.ac.be)... 146.175.13.81
Connecting to www.cnts.ua.ac.be (www.cnts.ua.ac.be)|146.175.13.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611540 (597K) [application/x-gzip]
Saving to: ‘train.txt.gz’

train.txt.gz        100%[===================>] 597.21K  1.36MB/s    in 0.4s    

2023-03-14 00:21:39 (1.36 MB/s) - ‘train.txt.gz’ saved [611540/611540]



In [ ]:
%%shell

cut -d ' ' -f 1,2 train.txt > train_processed.txt

In [ ]:
import random

# Open the original dataset file
with open('/content/drive/MyDrive/data/train_processed.txt', 'r') as f:
    data = f.read()

# Split the data by newline characters to get individual data points
data_points = data.strip().split('\n')

# Shuffle the data points randomly
random.shuffle(data_points)

# Calculate the index to split the data points
split_idx = int(len(data_points) * 0.8)

# Divide the data points into train and dev sets
train_data = data_points[:split_idx]
dev_data = data_points[split_idx:]

# Save the train and dev sets to separate files
with open('train_set.txt', 'w') as f:
    f.write('\n'.join(train_data))

with open('dev_set.txt', 'w') as f:
    f.write('\n'.join(dev_data))

In [ ]:
# Load the word embeddings from the Google News file
import gensim
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/data/GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
# Load train data
train_tags = []
with open('/content/drive/MyDrive/data/train_set.txt', 'r') as f:
    sentence = []
    for line in f:
        if line.strip() == '':
            for word_pos in sentence:
                pos_tag = word_pos.split()[1]
                train_tags.append(pos_tag)
            sentence = []
        else:
            sentence.append(line.strip())
    if sentence:
        for word_pos in sentence:
            pos_tag = word_pos.split()[1]
            train_tags.append(pos_tag)
            
# Print the unique set of POS
print(train_tags)

['.', 'NNP', 'PRP', 'NNP', 'JJ', 'IN', ',', 'NNP', 'NNP', 'VBN', 'IN', 'NNS', 'IN', 'NNP', 'IN', '.', 'VBD', 'DT', 'CC', '.', 'VB', 'NNP', 'DT', 'TO', '.', 'NN', 'IN', 'IN', '.', '.', 'VB', 'TO', 'IN', 'NNP', 'DT', 'NNP', 'JJR', 'CD', 'DT', 'IN', ',', 'NN', 'IN', 'RB', 'VBN', 'PRP', 'RB', 'NNS', 'IN', 'NNS', 'NNP', 'VBN', 'POS', ',', 'IN', 'NN', 'VBD', 'IN', 'VBN', 'NN', 'NN', ',', 'MD', 'DT', 'CC', 'CD', 'IN', 'NN', 'NN', 'DT', 'WRB', 'VBZ', 'VBD', 'NNP', 'NNS', 'IN', 'NNP', '.', 'PRP', 'NNP', 'DT', 'NN', 'NN', 'VBG', '$', 'DT', 'VB', 'NN', 'DT', 'DT', 'VB', 'NN', ',', 'NNS', 'VBN', 'NN', 'DT', 'DT', 'NNS', 'NN', 'IN', 'RB', '.', 'VBD', 'DT', 'VBN', 'IN', 'VBP', '.', 'NNS', 'NN', 'TO', 'TO', 'NNS', 'VB', 'VBZ', 'NN', 'NNS', 'PRP', ',', 'NN', 'NNS', 'NN', 'RB', 'NNP', 'NN', 'NNP', 'IN', 'NNP', 'NN', 'NN', 'VBZ', 'NN', 'PRP', 'NN', '.', 'IN', 'IN', 'JJ', '$', 'IN', 'JJ', 'CD', 'RB', 'DT', 'IN', 'IN', ',', 'NNP', 'VB', 'CD', 'NNP', '.', 'NNS', ',', 'NN', 'NNP', 'NNP', 'IN', 'NNP', 'PRP$'

In [ ]:
# Load train data
train_sentences = []
train_tags = []
with open('/content/drive/MyDrive/data/train_set.txt', 'r') as f:
    sentence = []
    tags = []
    for line in f:
        if line.strip() == '':
            train_sentences.append(sentence)
            train_tags.append(tags)
            sentence = []
            tags = []
        else:
            word, pos = line.strip().split()
            sentence.append(word)
            tags.append(pos)
    if sentence:
        train_sentences.append(sentence)
        train_tags.append(tags)

# Load dev data
dev_sentences = []
dev_tags = []
with open('/content/drive/MyDrive/data/dev_set.txt', 'r') as f:
    sentence = []
    tags = []
    for line in f:
        if line.strip() == '':
            dev_sentences.append(sentence)
            dev_tags.append(tags)
            sentence = []
            tags = []
        else:
            word, pos = line.strip().split()
            sentence.append(word)
            tags.append(pos)
    if sentence:
        dev_sentences.append(sentence)
        dev_tags.append(tags)

# Load the word embeddings from the Google News file
import gensim
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/data/GoogleNews-vectors-negative300.bin', binary=True)

# Define a function to extract features from a sentence
def extract_features(sentence):
    features = []
    for word in sentence:
        if word in word_vectors:
            features.append(word_vectors[word])
        else:
            features.append([0]*300)
    return features

# Extract features from train and dev data
train_features = [extract_features(sentence) for sentence in train_sentences]
dev_features = [extract_features(sentence) for sentence in dev_sentences]

# Flatten the features and tags for training
train_features_flat = [feature for sentence in train_features for feature in sentence]
train_tags_flat = [tag for sentence in train_tags for tag in sentence]


In [ ]:
# Train logistic regression model
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', max_iter=1000)
lr.fit(train_features_flat, train_tags_flat)




LogisticRegression(max_iter=1000, multi_class='multinomial', random_state=0)

In [ ]:
# Predict on dev set
dev_tags_pred = []
for sentence_features in dev_features:
    if sentence_features:
        sentence_tags_pred = lr.predict(sentence_features)
        dev_tags_pred.append(sentence_tags_pred)


In [ ]:
# Flatten the predicted tags for evaluation
dev_tags_flat = [tag for sentence in dev_tags for tag in sentence]
dev_tags_pred_flat = [tag for sentence_tags_pred in dev_tags_pred for tag in sentence_tags_pred]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(dev_tags_flat, dev_tags_pred_flat)
print('Accuracy on dev set:', accuracy)

Accuracy on dev set: 0.7517052562014681


In [ ]:
# Save the logistic regression model
import pickle
with open('/content/drive/MyDrive/data/lr_model.pkl', 'wb') as f:
    pickle.dump(lr, f)


In [ ]:
import joblib

# Load the model from file
model = joblib.load('/content/drive/MyDrive/data/lr_model.pkl')
